In [1]:
import pymongo
import pandas as pd
from pymongo import MongoClient

In [2]:
# Connection URI
client = MongoClient('mongodb+srv://<user>:<password>@cluster0.l3pqt.mongodb.net/MSA?retryWrites=true&w=majority')
# Select database
db = client['MSA']
# Select the collection within the database (in this case its GDP_raw data, check list below for other collections)
labor_force = db.LaborForce_raw
# Convert entire collection to Pandas dataframe
df = pd.DataFrame(list(labor_force.find()))
df

,_id,CBSA,series id,year,period,period_name,value
0,6036de026c3f5981ba4c55c7,11500,LAUMT011150000000006,2019,M12,December,46320
1,6036de026c3f5981ba4c55c8,11500,LAUMT011150000000006,2019,M11,November,46408
2,6036de026c3f5981ba4c55c9,11500,LAUMT011150000000006,2019,M10,October,46508
3,6036de026c3f5981ba4c55ca,11500,LAUMT011150000000006,2019,M09,September,45820
4,6036de026c3f5981ba4c55cb,11500,LAUMT011150000000006,2019,M08,August,46104
...,...,...,...,...,...,...,...
46075,6036de026c3f5981ba4d09c2,49340,LAUMT257960000000006,2010,M05,May,339705
46076,6036de026c3f5981ba4d09c3,49340,LAUMT257960000000006,2010,M04,April,339684
46077,6036de026c3f5981ba4d09c4,49340,LAUMT257960000000006,2010,M03,March,339383
46078,6036de026c3f5981ba4d09c5,49340,LAUMT257960000000006,2010,M02,February,339162


In [3]:
# drop unwanted columns
df.drop(columns=['_id', 'series id', 'period'], inplace=True)

In [4]:
print(df.dtypes)

CBSA            int64
year           object
period_name    object
value          object
dtype: object


In [5]:
# change data type
df[['year', 'value']] = df[['year', 'value']].apply(pd.to_numeric)
df.dtypes

CBSA            int64
year            int64
period_name    object
value           int64
dtype: object

In [6]:
df.rename(columns={'period_name':'month'}, inplace=True)

In [7]:
# load MSA dataset
df_msa = pd.read_csv('../everything.csv')
df_msa

,CBSA,MSA
0,10180,"Abilene, TX"
1,10420,"Akron, OH"
2,10500,"Albany, GA"
3,10540,"Albany-Lebanon, OR"
4,10580,"Albany-Schenectady-Troy, NY"
...,...,...
379,49420,"Yakima, WA"
380,49620,"York-Hanover, PA"
381,49660,"Youngstown-Warren-Boardman, OH-PA"
382,49700,"Yuba City, CA"


In [8]:
# combine dataframes to hold all relevant info
df = df.merge(df_msa, on='CBSA')
df

,CBSA,year,month,value,MSA
0,11500,2019,December,46320,"Anniston-Oxford, AL"
1,11500,2019,November,46408,"Anniston-Oxford, AL"
2,11500,2019,October,46508,"Anniston-Oxford, AL"
3,11500,2019,September,45820,"Anniston-Oxford, AL"
4,11500,2019,August,46104,"Anniston-Oxford, AL"
...,...,...,...,...,...
46075,49340,2010,May,339705,"Worcester, MA-CT"
46076,49340,2010,April,339684,"Worcester, MA-CT"
46077,49340,2010,March,339383,"Worcester, MA-CT"
46078,49340,2010,February,339162,"Worcester, MA-CT"


In [9]:
# reorder columns
df= df[['CBSA', 'MSA', 'year', 'month', 'value']]
df

,CBSA,MSA,year,month,value
0,11500,"Anniston-Oxford, AL",2019,December,46320
1,11500,"Anniston-Oxford, AL",2019,November,46408
2,11500,"Anniston-Oxford, AL",2019,October,46508
3,11500,"Anniston-Oxford, AL",2019,September,45820
4,11500,"Anniston-Oxford, AL",2019,August,46104
...,...,...,...,...,...
46075,49340,"Worcester, MA-CT",2010,May,339705
46076,49340,"Worcester, MA-CT",2010,April,339684
46077,49340,"Worcester, MA-CT",2010,March,339383
46078,49340,"Worcester, MA-CT",2010,February,339162


In [20]:
# create new collection to export clean data to mongo
labor_force_clean = db.LaborForce_clean

In [19]:
# turn dataframe into format for mongo to store
df_dict = df.to_dict(orient='records')

In [21]:
#labor_force_clean.insert_many(df_dict)